In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
import numpy as np
import numpy
import os
import math
import pandas as pd
import time

In [ ]:
# helper functions, partly taken/adapted from  https://github.com/slerch/ppnn

# CRPS loss from Rasp and Lerch (2018)
def crps_cost_function(y_true, y_pred):
    # Split input
    mu = y_pred[:, 0]
    sigma = y_pred[:, 1]
    y_true = y_true[:, 0]

    # To stop sigma from becoming negative we first have to 
    # convert it the the variance and then take the square
    # root again. 
    var = K.square(sigma)
    # The following three variables are just for convenience
    loc = (y_true - mu) / K.sqrt(var)
    phi = 1.0 / numpy.sqrt(2.0 * numpy.pi) * K.exp(-K.square(loc) / 2.0)
    Phi = 0.5 * (1.0 + tf.math.erf(loc / numpy.sqrt(2.0)))
    # First we will compute the crps for each input/target pair
    crps =  K.sqrt(var) * (loc * (2. * Phi - 1.) + 2 * phi - 1. / numpy.sqrt(numpy.pi))
    # Then we take the mean. The cost is now a scalar
    return K.mean(crps)

def normalize(data, method=None, shift=None, scale=None):
    result = numpy.zeros(data.shape)
    if method == "MINMAX":
        shift = numpy.min(data, axis=0)
        scale = numpy.max(data, axis=0) - numpy.min(data, axis=0) 
    elif method == "STD":
        shift = numpy.mean(data, axis=0)
        scale = numpy.std(data, axis=0)
    elif method == "MAX":
        scale = numpy.max(data, axis=0)
        shift = numpy.zeros(scale.shape)
    for index in range(len(data[0])):
        result[:,index] = (data[:,index] - shift[index]) / scale[index]
    return result, shift, scale

from scipy.stats import norm

# Note: needs to be adjusted if GaussianCRPS is used with exp-transformation
def crps_normal(mu, sigma, y):
    """
    Compute CRPS for a Gaussian distribution. 
    """
    # Make sure sigma is positive
    sigma = numpy.abs(sigma)
    loc = (y - mu) / sigma
    crps = sigma * (loc * (2 * norm.cdf(loc) - 1) + 
                    2 * norm.pdf(loc) - 1. / numpy.sqrt(numpy.pi))
    return crps

def save_ensemble(preds, exp_name, save=True):
    preds = numpy.array(preds)
    preds[:, :, 1] = numpy.abs(preds[:, :, 1])   # Make sure std is positive 
    mean_preds = numpy.mean(preds, 0)
    ens_score = crps_normal(mean_preds[:, 0], mean_preds[:, 1], testY).mean()
    # print(f'Ensemble test score = {ens_score}')
    if save:
        results_df = create_results_df(testDates[:,0], testIDs[:,0], mean_preds[:, 0], mean_preds[:, 1])
        results_df.to_csv(f'{exp_name}.csv')
    return(ens_score)

In [ ]:
# import non-AE data
dataRaw = numpy.load('.../data/ppnn.npy')

# remove soil moisture forecasts due to missing values
data = dataRaw[:,2:39] 

stations = numpy.genfromtxt('.../data/station_info.csv', delimiter=',', skip_header=1, usecols=[1,2,3,5,6])
stationsMap = {}
i = 0
for station in stations:
    stationsMap[int(station[0])] = numpy.concatenate(([i], station[1:]))
    i = i + 1
stationColumns = numpy.zeros((data.shape[0],4))
stationID = numpy.zeros((data.shape[0],1))
for t in range(stationColumns.shape[0]):
    stationColumns[t] = stationsMap[data[:,1][t]][1:]
    stationID[t] = stationsMap[data[:,1][t]][0]
       
obs = data[:,2]
# remove MJD from data
data = data[:,3:]

data_MJD = dataRaw[:,2].reshape((-1,1))

# output:
#    data: NWP input features
#    stationColumns: station-specific input features (lon, lat, altitude, orography)
#    data_MJD: date in MJD format

In [ ]:
# list of parameters to run simulations over all combinations of spatial inputs and encoding dimensions

AE_var_inputs_list = ["t2m", "all", "500gh", "850u", "850v", "t2m_gh500"]
AE_enc_dim_list = [2,4,8,12,16]
AE_early_stopped_list = [True]
hidden_layers_list = [2]
nodes_hidden_list = [100]
emb_dim_list = [15] 
epochs_list = [100]
early_stopping_list = [True]

In [ ]:
from itertools import product

def expand_grid(dictionary):
   return pd.DataFrame([row for row in product(*dictionary.values())], 
                       columns=dictionary.keys())

dictionary = {'AE_var_inputs': AE_var_inputs_list, 
              'AE_enc_dim': AE_enc_dim_list,
              'hidden_layers': hidden_layers_list,
              'nodes_hidden': nodes_hidden_list,
              'emb_dim': emb_dim_list,
              'epochs': epochs_list,
              'early_stopping': early_stopping_list}

par_list = expand_grid(dictionary)

In [ ]:
n_sim = par_list.shape[0]

In [ ]:
# load raw data
dataRaw = numpy.load('../data/ppnn.npy')

# remove soil moisture forecasts due to missing values
data = dataRaw[:,2:39] 

stations = numpy.genfromtxt('/home/sebastian/Projects/AE_postprocessing/local_tests/data/station_info.csv', delimiter=',', skip_header=1, usecols=[1,2,3,5,6])
stationsMap = {}
i = 0
for station in stations:
    stationsMap[int(station[0])] = numpy.concatenate(([i], station[1:]))
    i = i + 1
stationColumns = numpy.zeros((data.shape[0],4))
stationID = numpy.zeros((data.shape[0],1))
for t in range(stationColumns.shape[0]):
    stationColumns[t] = stationsMap[data[:,1][t]][1:]
    stationID[t] = stationsMap[data[:,1][t]][0]
       
obs = data[:,2]
# remove MJD from data
data = data[:,3:]

data_MJD = dataRaw[:,2].reshape((-1,1))

# output:
#    data: NWP input features
#    stationColumns: station-specific input features (lon, lat, altitude, orography)
#    data_MJD: date in MJD format

In [ ]:
# for loop to train all considered models

from tqdm.notebook import tqdm

for this_simulation in tqdm(range(n_sim)):
    
    # simulation parameters for this iteration
    this_AE_var_input = par_list['AE_var_inputs'][this_simulation]
    this_AE_variant = "simple"
    this_AE_enc_dim = par_list['AE_enc_dim'][this_simulation]
    this_AE_early_stopped = "True"
    this_hidden_layers = par_list['hidden_layers'][this_simulation]
    this_nodes_hidden = par_list['nodes_hidden'][this_simulation]
    this_emb_dim = par_list['emb_dim'][this_simulation]
    this_epochs = par_list['epochs'][this_simulation]
    this_early_stopping = par_list['early_stopping'][this_simulation]
    
    # load relevant AE data
    AE_base_dir = '.../AE_results/predictions/ConvAE_'
    
    fname_t2m = (AE_base_dir + 't2m_' + str(this_AE_enc_dim) + '.npy')
    fname_500gh = (AE_base_dir + '500gh_' + str(this_AE_enc_dim) + '.npy')
    fname_850u = (AE_base_dir + '850u_' + str(this_AE_enc_dim) + '.npy')
    fname_850v = (AE_base_dir + '850v_' + str(this_AE_enc_dim) + '.npy')
    
    data_AE_t2m_raw = numpy.load(fname_t2m)
    data_AE_500gh_raw = numpy.load(fname_500gh)  
    data_AE_850u_raw = numpy.load(fname_850u)
    data_AE_850v_raw = numpy.load(fname_850v)
    
    data_AE_t2m_repeated = np.repeat(data_AE_t2m_raw, repeats=537, axis=0)
    data_AE_500gh_repeated = np.repeat(data_AE_500gh_raw, repeats=537, axis=0)
    data_AE_850u_repeated = np.repeat(data_AE_850u_raw, repeats=537, axis=0)
    data_AE_850v_repeated = np.repeat(data_AE_850v_raw, repeats=537, axis=0)
    
    # remove missing values
    eval_start = 1764045
    train_end = 1763507 # train until 2015-12-30

    trainX_raw = data[:train_end,:]
    trainStationData = stationColumns[:train_end,:]
    trainY = obs[:train_end]
    trainDates = data_MJD[:train_end]
    trainIDs = stationID[:train_end]
    trainAEpreds_t2m = data_AE_t2m_repeated[:train_end,]
    trainAEpreds_500gh = data_AE_500gh_repeated[:train_end,]
    trainAEpreds_850u = data_AE_850u_repeated[:train_end,]
    trainAEpreds_850v = data_AE_850v_repeated[:train_end,]
    
    isnans = numpy.isnan(trainY)
    trainY = trainY[~isnans]
    trainX_raw = trainX_raw[~isnans]
    trainStationData = trainStationData[~isnans]
    trainDates = trainDates[~isnans]
    trainIDs = trainIDs[~isnans]
    trainAEpreds_t2m = trainAEpreds_t2m[~isnans]
    trainAEpreds_500gh = trainAEpreds_500gh[~isnans]
    trainAEpreds_850u = trainAEpreds_850u[~isnans]
    trainAEpreds_850v = trainAEpreds_850v[~isnans]

    testX_raw = data[eval_start:,:]
    testStationData = stationColumns[eval_start:,:]
    testY = obs[eval_start:]
    testDates = data_MJD[eval_start:]
    testIDs = stationID[eval_start:]
    testAEpreds_t2m = data_AE_t2m_repeated[eval_start:,]
    testAEpreds_500gh = data_AE_500gh_repeated[eval_start:,]
    testAEpreds_850u = data_AE_850u_repeated[eval_start:,]
    testAEpreds_850v = data_AE_850v_repeated[eval_start:,]

    isnans = numpy.isnan(testY)
    testY = testY[~isnans]
    testX_raw = testX_raw[~isnans]
    testStationData = testStationData[~isnans]
    testDates = testDates[~isnans]
    testIDs = testIDs[~isnans]
    testAEpreds_t2m = testAEpreds_t2m[~isnans]
    testAEpreds_500gh = testAEpreds_500gh[~isnans]
    testAEpreds_850u = testAEpreds_850u[~isnans]
    testAEpreds_850v = testAEpreds_850v[~isnans]
    
    # scale input features (except IDs and AE inputs)
    trainX, train_shift, train_scale = normalize(trainX_raw[:,:], method="MAX")
    trainStationData, train_shift_StationData, train_scale_StationData = normalize(trainStationData[:,:], 
                                                                                   method="MAX")
    
    testX = normalize(testX_raw[:,:], shift=train_shift, scale=train_scale)[0]
    testStationData = normalize(testStationData[:,:], 
                                shift=train_shift_StationData, 
                                scale=train_scale_StationData)[0]
    
    # combine AE inputs, depending on variant 
    if this_AE_var_input == "t2m":
        trainAEpreds_combined = trainAEpreds_t2m
        testAEpreds_combined = testAEpreds_t2m
    if this_AE_var_input == "500gh":
        trainAEpreds_combined = trainAEpreds_500gh
        testAEpreds_combined = testAEpreds_500gh
    if this_AE_var_input == "850u":
        trainAEpreds_combined = trainAEpreds_850u
        testAEpreds_combined = testAEpreds_850u
    if this_AE_var_input == "850v":
        trainAEpreds_combined = trainAEpreds_850v
        testAEpreds_combined = testAEpreds_850v
    if this_AE_var_input == "all":
        trainAEpreds_combined = np.concatenate((trainAEpreds_t2m, 
                                                trainAEpreds_500gh, 
                                                trainAEpreds_850u, 
                                                trainAEpreds_850v), 
                                               axis = 1)
        testAEpreds_combined = np.concatenate((testAEpreds_t2m, 
                                               testAEpreds_500gh, 
                                               testAEpreds_850u, 
                                               testAEpreds_850v), axis = 1)
    if this_AE_var_input == "t2m_gh500":
        trainAEpreds_combined = np.concatenate((trainAEpreds_t2m, 
                                                trainAEpreds_500gh), 
                                               axis = 1)
        testAEpreds_combined = np.concatenate((testAEpreds_t2m, 
                                               testAEpreds_500gh), axis = 1)

    
    # train ensemble of NN models in a loop
    
    start_time = time.time()
    
    nreps = 10
    trn_scores = []
    test_scores = []
    preds = []
    
    n_features = trainX.shape[1] # 34
    n_StationFeatures = trainStationData.shape[1] # 4
    emb_size = this_emb_dim
    n_hidden_layers = this_hidden_layers
    max_id = int(numpy.max([trainIDs.max(), testIDs.max()]))
    AE_enc_dim = trainAEpreds_combined.shape[1]

    hidden_nodes = this_nodes_hidden
    activation = 'relu' 
    optimizer='adam'
    lr = 0.002
    loss=crps_cost_function 
    n_outputs = 2
    this_epochs == 100:
    this_patience = 10
        
    for i in tqdm(range(nreps), leave=False):
        
        tf.compat.v1.reset_default_graph()
        keras.backend.clear_session()

        features_in = tf.keras.layers.Input(shape=(n_features,))
        stationdata_in = tf.keras.layers.Input(shape=(n_StationFeatures,))
        id_in = tf.keras.layers.Input(shape=(1,))
        AE_in = tf.keras.layers.Input(shape=(AE_enc_dim,))

        emb = tf.keras.layers.Embedding(max_id + 1, emb_size)(id_in)
        emb = tf.keras.layers.Flatten()(emb)

        x = tf.keras.layers.Concatenate()([features_in, stationdata_in, emb, AE_in])

        hidden_layers = np.repeat(hidden_nodes, n_hidden_layers)
        for h in hidden_layers:
            x = tf.keras.layers.Dense(h, activation=activation, kernel_regularizer=None)(x)
        x = tf.keras.layers.Dense(n_outputs, activation='linear', kernel_regularizer=None)(x)

        model = tf.keras.models.Model(inputs=[features_in, stationdata_in, id_in, AE_in], outputs=x)
        opt = tf.keras.optimizers.Adam(lr=lr)
        model.compile(loss=loss, optimizer=opt)
        
        if this_early_stopping == True:
            es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                           patience=this_patience, 
                                           restore_best_weights = True)
            model.fit([trainX, trainStationData, trainIDs, trainAEpreds_combined], 
                  trainY, 
                  epochs=this_epochs, 
                  batch_size=4096, 
                  verbose=0,
                  validation_split=1/9, 
                  callbacks=[es_callback])
        if this_early_stopping == False:
            model.fit([trainX, trainStationData, trainIDs, trainAEpreds_combined], 
                  trainY, 
                  epochs=this_epochs, 
                  batch_size=4096, 
                  verbose=0,
                  validation_split=1/9)
            
        elapsed_time = time.time() - start_time
            
        trn_scores.append(model.evaluate([trainX, trainStationData, trainIDs, trainAEpreds_combined], 
                                         trainY, 4096, 
                                         verbose=0))
        preds.append(model.predict([testX, testStationData, testIDs, testAEpreds_combined], 
                                   4096, verbose=0))
        # end for loop for NN model training
        
    # save predictions
    fname_save_preds = ('.../PP_results/predictions/ConvAE_' +
                       this_AE_var_input + '_' + 
                       str(this_AE_enc_dim) + '_' +
                       str(this_hidden_layers) + '_' +
                       str(this_nodes_hidden) + '_' +
                       str(this_emb_dim) + '_' +
                       str(this_epochs) + '_' +
                       str(this_early_stopping)
                      )                           
    this_test_score = save_ensemble(preds, fname_save_preds, save = True)